<a href="https://colab.research.google.com/github/youngmoo/ECES-434/blob/main/Class%208.2%20(2021-03-03).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ECES-434: Class 8.2 (2021-03-03)**
Week 8: It's already March!

In [ ]:
import numpy as np                      # NumPy, abbreviated as np
import matplotlib.pyplot as plt         # MatplotLib PyPlot module, abbreviated as plt
from matplotlib import animation, rc    # MatplotLib animation module
%matplotlib inline
from scipy import signal                # SciPy's signal module, for DSP functions
import soundfile as sf                  # Switching to the soundfile module for reading and writing soundfiles
import librosa

import IPython.display as ipd           # Interactive Python display module, for playing sounds
from IPython.display import HTML        # For displaying animations
rc('animation', html='jshtml')          # Provides animation controls

ClassPath = '/content/drive/My Drive/ECES-434 Sessions/Class 8-1/'

In [ ]:
rc('figure', figsize=(16,4))

In [ ]:
# CHANGE THIS to your Drexel username!!
username = 'anonymous'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Custom plotting functions
Because we're always plotting...

## plotSpectrogram

In [ ]:
def plotSpectrogram(sig, fs, win='hann', nseg=512, olap=256, fft_len=512):
  
  f1, t1, Sxx = signal.spectrogram(sig, fs, window=win, nperseg=nseg, noverlap=olap, nfft=fft_len)
  fig = plt.figure(figsize=(16,6))
  
  plt.pcolormesh(t1, f1, 20*np.log10(np.abs(Sxx)))
  plt.ylabel('Frequency (Hz)')
  plt.xlabel('Time (sec)')
  return plt

## myPlot(): properly formats time domain plot of a signal

In [ ]:
def myPlot(sig, fs=44100):
  fig = plt.figure(figsize=(16,4))
  t = np.arange(len(sig)) / fs
  plt.plot(t, sig)
  plt.xlabel('Time (sec)')
  return fig, plt

## myPlotFFT(): properly formats frequency domain plot of a signal

In [ ]:
def myPlotFFT(sig, n_fft=0, x_lim=22050, fs=44100):
  if n_fft==0:                 
    n_fft = len(sig)                    # Default to length of input signal
  S = np.fft.fft(sig, n_fft)
  N = len(S)
  f = np.arange(N) * fs / N
  fig = plt.figure(figsize=(16,6))
  plt.plot(f, 20*np.log10(np.abs(S)))
  plt.xlim(0, x_lim)
  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Magnitude (dB)')
  return fig, plt  

## myPlotFFTPhase

In [ ]:
def myPlotFFTPhase(sig, n_fft=0, x_lim=22050, fs=44100):
  if n_fft==0:                 
    n_fft = len(sig)                    # Default to length of input signal
  S = np.fft.fft(sig, n_fft)
  N = len(S)
  f = np.arange(N) * fs / N
  fig = plt.figure(figsize=(16,4))
  plt.plot(f, np.unwrap(np.angle(S)))
  plt.xlim(0, x_lim)
  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Phase (radians)')
  return fig, plt

## Custom FFT animation functions

In [ ]:
n_o = 0
f_size = 2048
n_hop = f_size / 2
N_fft = 4096
fs = 44100
this_f = np.arange(N_fft) * fs / N_fft

# First set up the figure, the axis, and the plot element we want to animate
def setupAnimFFT(x_lim=(0,20000), y_lim=(-120,100)):
  fig = plt.figure(figsize=(14,6))
  ax = plt.axes(xlim=x_lim,ylim=y_lim)
  plt.close()   # Don't output the final figure separately
  line, = ax.plot([], [])
  fig.tight_layout()
  return fig, line

# initialization function: plot the background of each frame
def initAnimFFT():
    line.set_data([], [])
    return (line,)

# animation function. This is called sequentially  
def animateFFT(i, sig):
    n1 = int(n_o + n_hop*i)
    n2 = int(n_o + n_hop*i + f_size)

    x_i = sig[n1:n2]
    X_i = np.fft.fft(x_i * np.hanning(len(x_i)), n=N_fft)
    X_mag = 20*np.log(np.abs(X_i))

    line.set_data(this_f, X_mag)
    return (line,)  

# Usage:
# fig, line = setupAnimFFT()
# anim = animation.FuncAnimation(fig, animateFFT, init_func=initAnimFFT, frames=120, fargs=(signal,), interval=1000/30, blit=True)
# anim

# Load sound file...
*Chariots of Fire* by Vangelis (1981)

In [ ]:
cof_s, fs = sf.read(ClassPath + 'Chariots-44kHz.wav')
cof = np.mean(cof_s,axis=1)
ipd.Audio(cof,rate=fs)

In [ ]:
nFFT = 2048
nHop = 512

COF = librosa.stft(cof,n_fft=nFFT, hop_length=nHop, win_length=1024, window='hann')

nBins, nFrames = np.shape(COF)
f1 = np.arange(nBins)/(nFFT/2) * (fs/2)
t1 = np.arange(nFrames) * nHop / fs

(nBins, nFrames)

In [ ]:
plt.plot(f1)
plt.ylim(4000,8000)
plt.xlim(186,371)
f1[371]

In [ ]:
plt.plot(f1)
plt.ylim(8000,16000)
plt.xlim(372,744)
f1[744]

In [ ]:
f_c1 = 4000
f_c2 = 8000

tri_f1 = np.zeros(nBins)
tri_f1[:186] = f1[:186] /(f_c1)
tri_f1[186:372] = 1 - (f1[186:372]- f_c1)/f_c1


tri_f2 = np.zeros(nBins)
tri_f2[186:372] = (f1[186:372] - f_c1)/(f_c1)
tri_f2[372:744] = 1 - (f1[372:744]- f_c2)/f_c2

plt.plot(f1, tri_f1, f1, tri_f2)
#plt.xlim(0,4000)

In [ ]:
# Need this in order to tile it for the full STFT matrix
tri_f1.shape = (nBins,1)
tri_f2.shape = (nBins,1)

H1 = np.tile(tri_f1, (1,nFrames)) 
H2 = np.tile(tri_f2, (1,nFrames)) 
plt.pcolormesh(t1,f1, H1)

In [ ]:
COF_H1 = COF * H1
COF_H2 = COF * H2

plt.pcolormesh(t1, f1, 20*np.log10(np.abs(COF_H2)))
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (sec)')

In [ ]:
cof_h1 = librosa.istft(COF_H1, hop_length=nHop, win_length=1024)
ipd.Audio(cof_h1, rate=fs)

In [ ]:
cof_h2 = librosa.istft(COF_H2, hop_length=nHop, win_length=1024)
ipd.Audio(cof_h2, rate=fs)

In [ ]:
ipd.Audio(cof_h1 + cof_h2, rate=fs)

In [ ]:
COF_H2p = np.zeros((nBins,nFrames) )

nPeaks = 5

for n in range(nFrames):
  sort_idx = np.argsort( np.abs(COF_H2[:,n]) )
  peak_idxs = sort_idx[-nPeaks:len(sort_idx)]
  COF_H2p[peak_idxs, n] = COF_H[peak_idxs, n]

plt.pcolormesh(t1, f1, 20*np.log10(np.abs(COF_H2p)))
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (sec)')

In [ ]:
cof_h2p = librosa.istft(COF_H2p, hop_length=nHop, win_length=1024)
ipd.Audio(cof_h2p, rate=fs)

In [ ]:
h2n = np.sum(np.abs(COF_H2 - COF_H2p), axis=0) / 6000
np.shape(h2n)
plt.plot(h2n)

In [ ]:
noiz = 2*np.random.rand(len(cof_h2)) - 1
NOIZ = librosa.stft(noiz)

h2n.shape = (1,nFrames)
H2N = np.tile(h2n, (1025,1))

H2_env = H2 * H2N

COF_H2N_env = NOIZ * H2_env
plt.pcolormesh(t1, f1, 20*np.log10(np.abs(COF_H2N_env)))
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (sec)')

In [ ]:
cof_h2_env = librosa.istft(COF_H2N_env, hop_length=nHop, win_length=1024)
ipd.Audio(cof_h2_env, rate=fs)

In [ ]:
ipd.Audio(cof_h2_env + cof_h2p + cof_h1, rate=fs)

In [ ]:
cof_p = librosa.istft(COF_P, hop_length=nHop, win_length=1024)
ipd.Audio(cof_p, rate=fs)

In [ ]:
COF_H1p = np.zeros((nBins,nFrames) )

nPeaks = 128

for n in range(nFrames):
  sort_idx = np.argsort( np.abs(COF_H1[:,n]) )
  peak_idxs = sort_idx[-nPeaks:len(sort_idx)]
  COF_H1p[peak_idxs, n] = COF_H[peak_idxs, n]

plt.pcolormesh(t1, f1, 20*np.log10(np.abs(COF_H1p)))
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (sec)')

In [ ]:
cof_h1p = librosa.istft(COF_H1p, hop_length=nHop)
ipd.Audio(cof_h1p, rate=fs)

In [ ]:
h1n = np.sum(np.abs(COF_H1), axis=0) / 4000
np.shape(h1n)
plt.plot(h1n)

In [ ]:
h1n.shape = (1,nFrames)
H1N = np.tile(h1n, (1025,1))

H1_env = H1 * H1N

COF_H1N_env = NOIZ * H1_env
plt.pcolormesh(t1, f1, 20*np.log10(np.abs(COF_H1N_env)))
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (sec)')

In [ ]:
cof_h1_env = librosa.istft(COF_H1N_env)
ipd.Audio(cof_h1_env, rate=fs)